# Functions as First Class Object

In [1]:
from operator import mul
from functools import reduce
from functools import partial
from operator import methodcaller
from collections import namedtuple, Counter
from collections.abc import Sequence, Iterable
from typing import Optional, Union, TypeVar, Callable

In [2]:
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)


print(factorial.__doc__)
print(type(factorial))

fact = factorial
print("\n", fact, sep='')
print(map(factorial, range(11)))
print(list(map(factorial, range(11))))

None
<class 'function'>

<function factorial at 0x00000290D42B7640>
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


In [3]:
print([callable(obj) for obj in (abs, str, 'Ni')])


[True, True, False]


In [4]:
def tag(name, *content, class_=None, **attrs):
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value
                  in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    # print(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>'
                    for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'


print(tag('br'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', class_='sidebar'))
print(tag(content='testing', name='img'))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 
          'src': 'sunset.jpg', 'class': 'framed'}
print(tag(**my_tag))
print(tag(class_='sidebar', **my_tag))

<br />
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />
<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />
<img class="sidebar" src="sunset.jpg" title="Sunset Boulevard" />


In [5]:
def f(a, *, b):
    return a, b


try:
    print(f(1, 2))
    print('f(1, 2)')
except:
    print(f(1, b=2))
    print('f(1, b=2).\tNot f(1, 2)')

(1, 2)
f(1, b=2).	Not f(1, 2)


In [6]:
def divmod(a, b, /):
    return (a // b, a % b)


try:
    print(divmod(a=10, b=3))
    print('divmod(a=10, b=3)')
except:
    print(divmod(10, 3))
    print('divmod(10, 3).\tNot divmod(a=10, b=3)')

(3, 1)
divmod(10, 3).	Not divmod(a=10, b=3)


In [7]:
def factorial(n):
    return reduce(lambda a, b: a * b, range(1, n + 1))
    # Or: return reduce(mul, range(1, n + 1))


s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
hyphenate = methodcaller('replace', ' ', '-')
print(hyphenate(s))
print(str.upper(s))


triple = partial(mul, 3)
print("\n", triple(7), sep="")
print(list(map(triple, range(1, 10))))

THE TIME HAS COME
The-time-has-come
THE TIME HAS COME

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [8]:
plural: Optional[str] = None    # before
plural: Union[str, None]
plural: str | None = None       # python 3.10

point = namedtuple('point', 'x y')
p = point(2, 3)
print(p.x, p.y, type(p))

2 3 <class '__main__.point'>


In python3.7 and 3.8
But it does not work in 3.6 or earlier

```python
from __future__ import annotations

def tokenize(text: str) -> list[str]:
    return text.upper().split()

```

In [9]:
def columnize(
        sequence: Sequence[str], num_columns: int = 0
) -> list[tuple[str, ...]]:
    if num_columns == 0:
        num_columns = round(len(sequence) ** 0.5)
    num_rows, reminder = divmod(len(sequence) ,num_columns)
    num_rows += bool(reminder)
    return [tuple(sequence[i::num_rows]) for i in range(num_rows)]

In [10]:
FromTo = tuple[str, str]


def zip_placer(text: str, changes: Iterable[FromTo]) -> str:
    for from_, to in changes:
        text = text.replace(from_, to)
    return text


l33t = [('a', '4'), ('e', '3'), ('i', '1'), ('o', '0')]
text = 'mad skilled noob powned leet'
print(zip_placer(text, l33t))

m4d sk1ll3d n00b p0wn3d l33t


In [11]:
T = TypeVar('T')


def mode(data: Iterable[T]) -> T:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs


print(mode([1, 1, 2, 3, 3, 5, 3, 3, 4]))

[(3, 4)]


In [12]:
def add(a: int, b: int) -> int:
    return a + b


def process(func: Callable[[int, int], int], a: int, b: int) -> int:
    return func(a, b)


result = process(add, 2, 3)
print(result)  # Output: 5
print(hex(result))

5
0x5


In [13]:
def tag_v1(
        num1, 
        name: str,
        /,
        *content: str,
        class_: Optional[str] = None,
        num2, 
        **attrs: str,
) -> str: pass

The type of the `content` local variable will be `tuple[str, ...]`
The type of the `attrs` will be `dict[str, str]`. 
For a type hint lisk `**attrs: float`, the type of the `attrs` would be `dict[str, float]`. 